In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

from math import sqrt 
import lightgbm as lgb
from sklearn.metrics import mean_squared_error 
from sklearn.model_selection import KFold, train_test_split
import requests
from io import StringIO 
%matplotlib inline
pd.set_option("display.max_rows", 100)
pd.set_option("display.max_columns", 100)

In [0]:
train  = pd.read_csv("/content/drive/My Drive/qdcomp/trainaway.csv")
test  = pd.read_csv("/content/drive/My Drive/qdcomp/testaway.csv")

In [0]:
test.drop("target",axis  = 1,inplace  = True)

In [0]:
train.shape,test.shape

((15539, 771), (5035, 770))

In [0]:
train.head()

,ID,location,target,newtemp0,newprecip0,newrel_humidity0,newwind_dir0,windspeed0,atmospherepressure0,newtemp1,newprecip1,newrel_humidity1,newwind_dir1,windspeed1,atmospherepressure1,newtemp2,newprecip2,newrel_humidity2,newwind_dir2,windspeed2,atmospherepressure2,newtemp3,newprecip3,newrel_humidity3,newwind_dir3,windspeed3,atmospherepressure3,newtemp4,newprecip4,newrel_humidity4,newwind_dir4,windspeed4,atmospherepressure4,newtemp5,newprecip5,newrel_humidity5,newwind_dir5,windspeed5,atmospherepressure5,newtemp6,newprecip6,newrel_humidity6,newwind_dir6,windspeed6,atmospherepressure6,newtemp7,newprecip7,newrel_humidity7,newwind_dir7,windspeed7,...,windspeed119,atmospherepressure119,newtemp120,newprecip120,newrel_humidity120,newwind_dir120,windspeed120,atmospherepressure120,max_temp,min_temp,mean_temp,std_temp,var_temp,median_temp,ptp_temp,max_precip,min_precip,mean_precip,std_precip,var_precip,median_precip,ptp_precip,max_rel_humidity,min_rel_humidity,mean_rel_humidity,std_rel_humidity,var_rel_humidity,median_rel_humidity,ptp_rel_humidity,max_wind_dir,min_wind_dir,mean_wind_dir,std_wind_dir,var_wind_dir,median_wind_dir,ptp_wind_dir,max_wind_spd,min_wind_spd,mean_wind_spd,std_wind_spd,var_wind_spd,median_wind_spd,ptp_wind_spd,max_atmos_press,min_atmos_press,mean_atmos_press,std_atmos_press,var_atmos_press,median_atmos_press,ptp_atmos_press
0,ID_train_0,C,45.126304,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.673333,87.750833,19.583333,0.000,0.874583,325.478885,0.855000,87.768333,27.208333,19.275000,22.299527,2.594011,6.728895,21.358333,7.933333,0.000,0.0,0.000000,0.000000,0.000000,0.0,0.000,0.891500,0.510818,0.758530,0.120234,0.014456,0.806292,0.380682,345.726333,6.535299,209.574655,120.231802,14455.686315,220.039988,339.191034,1.517500,0.162500,0.677348,0.394041,0.155268,0.666250,1.355000,87.871667,87.614167,87.752434,0.070374,0.004952,87.762083,0.257500
1,ID_train_1,D,79.131702,22.533333,0.102,0.744583,281.664310,2.377500,90.320000,21.716667,0.0,0.808083,89.156293,1.126667,90.377500,20.833333,0.0,0.911167,81.968539,0.700833,90.440833,20.983333,0.000,0.916333,291.018632,0.341667,90.472500,20.875000,0.000,0.929750,279.391524,0.440833,90.454167,20.141667,0.0,0.954000,158.026892,0.380000,90.394167,19.375000,0.0,0.964083,248.316596,0.467500,90.333333,18.791667,0.034,0.970750,211.143363,0.4625,...,1.475833,90.439167,21.450000,0.561,0.797333,296.967254,1.019167,90.529167,33.616667,17.983333,24.679063,4.266955,18.206903,23.791667,15.633333,0.561,0.0,0.007025,0.051706,0.002673,0.0,0.561,0.979250,0.403083,0.758058,0.160952,0.025906,0.791000,0.576167,357.730286,3.543198,177.109855,96.802929,9370.807133,153.164843,354.187088,3.009167,0.248333,0.899208,0.539199,0.290736,0.744167,2.760833,90.725000,90.056667,90.429924,0.156000,0.024336,90.429167,0.668333
2,ID_train_10,A,32.661304,28.975000,0.000,0.573333,NaN,NaN,88.551667,27.950000,0.0,0.597167,NaN,NaN,88.464167,29.600000,0.0,0.566833,NaN,NaN,88.319167,26.425000,0.102,0.627667,NaN,NaN,88.240000,22.091667,0.136,0.755417,NaN,NaN,88.230000,21.775000,0.0,0.777417,NaN,NaN,88.268333,22.333333,0.0,0.787000,NaN,NaN,88.370000,21.950000,2.160,0.831917,NaN,NaN,...,1.481667,88.475833,30.091667,0.000,0.437917,250.105258,1.746667,88.415833,31.841667,18.458333,24.112317,3.776377,14.261020,23.304167,13.383333,7.804,0.0,0.151767,0.798401,0.637445,0.0,7.804,0.930333,0.376750,0.712944,0.168129,0.028267,0.766458,0.553583,357.265773,0.581510,201.373211,92.105195,8483.367004,208.901301,356.684263,2.751667,0.331667,1.038656,0.585221,0.342484,0.942500,2.420000,88.813333,87.982500,88.420931,0.179481,0.032213,88.425000,0.830833
3,ID_train_100,A,53.850238,22.966667,0.000,0.843083,300.085057,1.446667,88.615000,24.266667,0.0,0.790250,293.676960,1.192500,88.530833,25.275000,0.0,0.737500,294.517465,1.324167,88.400000,25.625000,0.000,0.728000,301.921417,1.544167,88.271667,25.866

In [0]:
train.columns

Index(['ID', 'location', 'target', 'newtemp0', 'newprecip0',
       'newrel_humidity0', 'newwind_dir0', 'windspeed0', 'atmospherepressure0',
       'newtemp1',
       ...
       'var_wind_spd', 'median_wind_spd', 'ptp_wind_spd', 'max_atmos_press',
       'min_atmos_press', 'mean_atmos_press', 'std_atmos_press',
       'var_atmos_press', 'median_atmos_press', 'ptp_atmos_press'],
      dtype='object', length=771)

In [0]:
from sklearn.preprocessing import LabelEncoder
lab  = LabelEncoder()

In [0]:
lab.fit(train["location"])

LabelEncoder()

In [0]:
train.location  = lab.transform(train.location)
test.location  = lab.transform(test.location)

In [0]:
train.target.min(),train.target.max()

(1.4526190476190477, 475.82)

In [0]:
!pip install catboost

     |████████████████████████████████| 64.4MB 51kB/s 


In [0]:
X=train.drop(["ID",'target'],axis =1)
y = train.target

In [0]:
test_id  = test['ID']

In [0]:
from catboost import CatBoostRegressor
errcb2=[]
y_pred_totcb2=[]
from sklearn.model_selection import KFold,StratifiedKFold, TimeSeriesSplit
from sklearn.metrics import mean_squared_error
fold=KFold(n_splits=20)#15#5#10
i=1
for train_index, test_index in fold.split(X,y):
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    m2  = CatBoostRegressor(   n_estimators=500,eval_metric='RMSE',learning_rate=0.2, random_seed= 1234, use_best_model=True )
    m2.fit(X_train,y_train,eval_set=[(X_train,y_train),(X_test, y_test)], early_stopping_rounds=200,verbose=100,)#erly100
    preds=m2.predict(X_test)
    print("err: ",np.sqrt(mean_squared_error(y_test,preds)))
    errcb2.append(np.sqrt(mean_squared_error(y_test,preds)))
    p2 = m2.predict(test.drop(["ID"],axis = 1))
    y_pred_totcb2.append(p2)
np.mean(errcb2)

0:	learn: 40.2026142	test: 40.2026142	test1: 40.7895659	best: 40.7895659 (0)	total: 255ms	remaining: 2m 7s
100:	learn: 23.2279571	test: 23.2279571	test1: 26.6324761	best: 26.6324761 (100)	total: 25.7s	remaining: 1m 41s
200:	learn: 18.9783741	test: 18.9783741	test1: 25.5322833	best: 25.4920203 (197)	total: 50.6s	remaining: 1m 15s
300:	learn: 15.8883770	test: 15.8883770	test1: 25.1563587	best: 25.1247230 (291)	total: 1m 15s	remaining: 50.1s
400:	learn: 13.5195754	test: 13.5195754	test1: 24.7509073	best: 24.7509073 (400)	total: 1m 41s	remaining: 24.9s
499:	learn: 11.7130594	test: 11.7130594	test1: 24.4303418	best: 24.4290532 (498)	total: 2m 5s	remaining: 0us

bestTest = 24.42905316
bestIteration = 498

Shrink model to first 499 iterations.
err:  24.429053325611573
0:	learn: 40.3193503	test: 40.3193503	test1: 37.4711715	best: 37.4711715 (0)	total: 265ms	remaining: 2m 12s
100:	learn: 23.1046525	test: 23.1046525	test1: 25.0609496	best: 25.0507217 (99)	total: 25.8s	remaining: 1m 41s
200:	lear

25.38976689679425

In [0]:
np.mean(errcb2)

25.38976689679425

In [0]:
d = {'ID': test_id, 'target': np.mean(y_pred_totcb2, 0)}
sub = pd.DataFrame(data=d)
sub = sub[['ID', 'target']]

In [0]:
sub.target.min(),sub.target.max()

(15.885486421092144, 234.75770129928296)

In [0]:
sub.to_csv("fresh.csv",index  = False)

In [0]:
from google.colab import files

In [0]:
files.download("fresh.csv")